<a href="https://colab.research.google.com/github/Junior11995/Challenge-Telecom-X-an-lisis-de-evasi-n-de-clientes/blob/main/desaf_o_telecom_x.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>